# Description

According to the settings specified below, this notebook:
 1. reads all the data from one source (GTEx, recount2, etc) according to the gene selection method (`GENE_SELECTION_STRATEGY`),
 2. runs a quick performance test using the correlation coefficient specified (`CORRELATION_METHOD`), and
 3. computes the correlation matrix across all the genes using the correlation coefficient specified.

# Modules

In [1]:
from time import time

import pandas as pd
from tqdm import tqdm

from ccc import conf
from ccc.utils import simplify_string
from ccc.corr import spearman

# Settings

In [2]:
GENE_SELECTION_STRATEGY = "var_pc_log2"

In [ ]:
# select the top 5 tissues (according to sample size, see nbs/05_preprocessing/00-gtex_v8-split_by_tissue.ipynb)
TISSUES = [
    # "Muscle - Skeletal",
    "Whole Blood",
    # "Skin - Sun Exposed (Lower leg)",
    # "Adipose - Subcutaneous",
    # "Artery - Tibial",
]

In [3]:
CORRELATION_METHOD = spearman

method_name = CORRELATION_METHOD.__name__
display(method_name)

'spearman'

In [4]:
PERFORMANCE_TEST_N_TOP_GENES = 500

# Paths

In [5]:
INPUT_DIR = conf.GTEX["GENE_SELECTION_DIR"]
display(INPUT_DIR)

assert INPUT_DIR.exists()

PosixPath('/opt/data/results/gtex_v8/gene_selection')

In [6]:
OUTPUT_DIR = conf.GTEX["SIMILARITY_MATRICES_DIR"]
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_DIR)

PosixPath('/opt/data/results/gtex_v8/similarity_matrices')

# Data loading

In [7]:
tissue_in_file_names = [f"_data_{simplify_string(t.lower())}-" for t in TISSUES]

54

[PosixPath('/opt/data/results/gtex_v8/gene_selection/gtex_v8_data_adipose_subcutaneous-var_pc_log2.pkl'),
 PosixPath('/opt/data/results/gtex_v8/gene_selection/gtex_v8_data_adipose_visceral_omentum-var_pc_log2.pkl'),
 PosixPath('/opt/data/results/gtex_v8/gene_selection/gtex_v8_data_adrenal_gland-var_pc_log2.pkl'),
 PosixPath('/opt/data/results/gtex_v8/gene_selection/gtex_v8_data_artery_aorta-var_pc_log2.pkl'),
 PosixPath('/opt/data/results/gtex_v8/gene_selection/gtex_v8_data_artery_coronary-var_pc_log2.pkl')]

In [ ]:
input_files = sorted(list(INPUT_DIR.glob(f"*-{GENE_SELECTION_STRATEGY}.pkl")))
input_files = [
    f for f in input_files if any(tn in f.name for tn in tissue_in_file_names)
]
display(len(input_files))

assert len(input_files) == len(TISSUES), len(TISSUES)
display(input_files)

# Compute similarity

## Performance test

In [8]:
display(input_files[0])
test_data = pd.read_pickle(input_files[0]).iloc[:PERFORMANCE_TEST_N_TOP_GENES]

PosixPath('/opt/data/results/gtex_v8/gene_selection/gtex_v8_data_adipose_subcutaneous-var_pc_log2.pkl')

In [9]:
test_data.shape

(500, 663)

In [10]:
test_data.head()

,GTEX-1117F-0226-SM-5GZZ7,GTEX-111CU-1826-SM-5GZYN,GTEX-111FC-0226-SM-5N9B8,GTEX-111VG-2326-SM-5N9BK,GTEX-111YS-2426-SM-5GZZQ,GTEX-1122O-2026-SM-9YFMG,GTEX-1128S-2126-SM-5H12U,GTEX-113IC-0226-SM-5HL5C,GTEX-117YX-2226-SM-5EGJJ,GTEX-11DXW-0326-SM-5H11W,...,GTEX-ZXES-2026-SM-5NQ6R,GTEX-ZXG5-0226-SM-59HJI,GTEX-ZYFC-0326-SM-5NQ7H,GTEX-ZYFD-0226-SM-5NQ86,GTEX-ZYT6-0326-SM-7LG5R,GTEX-ZYVF-0226-SM-5GIEG,GTEX-ZYW4-0226-SM-5E44M,GTEX-ZYY3-0226-SM-5E45M,GTEX-ZZ64-1626-SM-5E43W,GTEX-ZZPU-2726-SM-5NQ8O
gene_ens_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000129824.15,0.9301,270.4000,345.8000,242.20,180.800,0.1904,0.18640,294.40,332.6000,216.0000,...,0.2916,311.700,230.0000,302.20000,316.500,0.2567,340.2000,0.6962,269.1000,0.1917
ENSG00000149968.11,100.4000,785.1000,14.0000,44.48,17.960,2.5670,0.18410,46.91,13.9500,0.1953,...,3.9480,108.800,3.8230,6.61700,11.440,7.6190,27.4300,21.0200,122.6000,10.8200
ENSG00000134184.12,75.6800,0.2281,0.2070,0.00,36.930,0.1651,0.06058,174.00,0.1429,56.4700,...,54.9200,46.230,0.1468,0.04705,67.240,0.4005,0.5507,56.0600,0.4048,0.0000
ENSG00000224114.1,1.1500,0.8068,0.5856,74.28,0.314,0.6115,49.95000,139.60,83.5400,1.3640,...,0.1915,5.593,60.0200,61.04000,1.596,81.6000,149.4000,0.2032,1.2270,0.5036
ENSG00000173432.10,2765.0000,241.1000,1.5220,99.98,921.400,2859.0000,3874.00000,21.41,1338.0000,61.4800,...,1666.0000,165.300,81.9700,180.90000,24.650,276.2000,53.5400,1436.0000,140.5000,756.3000


This is a quick performance test of the correlation measure. The following line (`_tmp = ...`) is the setup code, which is needed in case the correlation method was optimized using `numba` and needs to be compiled before performing the test.

In [11]:
_tmp = CORRELATION_METHOD(test_data.iloc[:3])

display(_tmp.shape)
display(_tmp)

(3, 3)

gene_ens_id,ENSG00000129824.15,ENSG00000149968.11,ENSG00000134184.12
gene_ens_id,,,
ENSG00000129824.15,1.000000,0.404501,0.016874
ENSG00000149968.11,0.404501,1.000000,-0.062548
ENSG00000134184.12,0.016874,-0.062548,1.000000


In [12]:
%timeit -r1 CORRELATION_METHOD(test_data)

68.4 ms ± 318 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Run

In [13]:
pbar = tqdm(input_files, ncols=100)

for tissue_data_file in pbar:
    pbar.set_description(tissue_data_file.stem)

    # read
    data = pd.read_pickle(tissue_data_file)

    # compute correlations
    start_time = time()

    data_corrs = CORRELATION_METHOD(data)

    end_time = time()
    elapsed_time = end_time - start_time
    display(elapsed_time)

    # save
    output_filename = f"{tissue_data_file.stem}-{method_name}.pkl"
    data_corrs.to_pickle(path=OUTPUT_DIR / output_filename)

gtex_v8_data_whole_blood-var_pc_log2: 100%|█████████████████████████| 54/54 [02:41<00:00,  2.99s/it]
